In [ ]:
pwd

In [ ]:
pip install boto3 sagemaker transformers

In [ ]:
pip install --upgrade accelerate


In [ ]:
from PIL import Image
import requests

from transformers import CLIPProcessor, CLIPModel, CLIPTokenizer

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
model.save_pretrained("./clip_mod2")
processor.save_pretrained("./clip_mod2")


In [ ]:
import torch
import os
save_path = "clip_mod2"
torch.save(model.state_dict(), os.path.join(save_path, "pytorch_model.bin"))

In [ ]:
cd clip_mod2

In [ ]:
!rm -r model.safetensors

In [ ]:
!tar czvf model.tar.gz *

In [ ]:
import boto3

s3 = boto3.client('s3')
bucket_name = "your-bucket"
model_file = "model.tar.gz"
s3.upload_file(model_file, bucket_name, "clip/model.tar.gz")



In [ ]:
import sagemaker
from sagemaker.huggingface import HuggingFaceModel

# Initialize the SageMaker session
sagemaker_session = sagemaker.Session()
import sagemaker
from sagemaker import get_execution_role

# Get the SageMaker execution role
role = get_execution_role()

print(f"Execution role: {role}")


huggingface_model = HuggingFaceModel(
    model_data= "s3://your-bucket/clip/model.tar.gz",
    transformers_version='4.26',
	pytorch_version='1.13',
	py_version='py39',
	role=role,
    
)


In [ ]:
predictor = huggingface_model.deploy(
	initial_instance_count=1, # number of instances
	instance_type='ml.m5.xlarge' # ec2 instance type
)
